In [2]:
# Fix randomness and hide warnings
seed = 73

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [3]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
# tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.12.0


In [4]:
# Import other libraries
import cv2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [35]:
#load data
data = np.load('/kaggle/input/clean-dataset/clean_dataset.npz', allow_pickle=True)


X = data["data"]
Y = data["labels"]
X.shape, Y.shape

((5004, 96, 96, 3), (5004,))

In [36]:
X = (X/255).astype(np.float32)

#change labels in 0 for healthy and 1 for unhealthy

for i in range(Y.size):
  if Y[i] == "healthy":
    Y[i] = 0
  else:
    Y[i] = 1

Y

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [37]:
idx = np.random.permutation(len(Y))
X,Y = X[idx], Y[idx]

In [9]:
#train, validation, test split (80,10,10)

#one-hot encoding
y = tfk.utils.to_categorical(Y,2)

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=520, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=520, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (3964, 96, 96, 3), y_train shape: (3964, 2)
X_val shape: (520, 96, 96, 3), y_val shape: (520, 2)
X_test shape: (520, 96, 96, 3), y_test shape: (520, 2)


In [21]:
from tensorflow.keras.applications import EfficientNetV2M
model = EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    pooling='avg',
)

model.trainable = False

## Model with data augmentation

In [8]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
model.trainable = False

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))

preprocessing = tf.keras.Sequential([
    tfkl.RandomTranslation(height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3)),
    tfkl.RandomFlip(mode="horizontal_and_vertical", seed=None),
], name='preprocessing')

preprocessing = preprocessing(inputs)


# Connect EfficientNet to the input
x = model(preprocessing)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Define optimizer, loss, and metrics
# AdamW is an Adam optimizer which applies weight_decay to network layers,
# i.e it's another way to apply l2 regularization to the whole network
optimizer = tfk.optimizers.AdamW(1e-4, weight_decay=5e-4)
loss = tfk.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Display model summary
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 53,152,950
Trainable params: 2,562
Non-trainable params: 53,150,388
_________________________________________________________________


## Model without data augmentation

In [ ]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
model.trainable = False

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))


# Connect EfficientNet to the input
x = model(inputs)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Define optimizer, loss, and metrics
# AdamW is an Adam optimizer which applies weight_decay to network layers,
# i.e it's another way to apply l2 regularization to the whole network
optimizer = tfk.optimizers.AdamW(1e-4, weight_decay=5e-4)
loss = tfk.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Display model summary
tl_model.summary()

## Model with added dense + batch norm + dropout + augmentation

In [21]:
# Use the supernet as feature extractor, i.e. freeze all its weigths
model.trainable = False

# Create an input layer with shape (96, 96, 3)
inputs = tfk.Input(shape=(96, 96, 3))

preprocessing = tf.keras.Sequential([
    tfkl.RandomTranslation(height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3)),
    tfkl.RandomFlip(mode="horizontal_and_vertical", seed=None),
], name='preprocessing')

preprocessing = preprocessing(inputs)


# Connect EfficientNet to the input
x = model(preprocessing)

x = tfkl.Dense(64, activation='swish')(x)

x = tfkl.BatchNormalization(name='BatchNorm0')(x)

# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Define optimizer, loss, and metrics
# AdamW is an Adam optimizer which applies weight_decay to network layers,
# i.e it's another way to apply l2 regularization to the whole network
optimizer = tfk.optimizers.AdamW(1e-4, weight_decay=5e-4)
loss = tfk.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
tl_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Display model summary
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 64)                81984     
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 64)               256       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130   

## Building the model with hyperparameter tuning

In [22]:
def build_model(hp):
    output_dense_layer_units = hp.Int("units", min_value=32, max_value=512, step=32)
    dropout = hp.Boolean("dropout")
    dropout_rate = hp.Float("droprate", min_value=0.2, max_value=0.3, sampling="linear")
    
    # Create an input layer with shape (96, 96, 3)
    inputs = tfk.Input(shape=(96, 96, 3))

    preprocessing = tf.keras.Sequential([
        tfkl.RandomTranslation(height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3)),
        tfkl.RandomFlip(mode="horizontal_and_vertical", seed=None),
    ], name='preprocessing')

    preprocessing = preprocessing(inputs)


    # Connect EfficientNet to the input
    x = model(preprocessing)

    x = tfkl.Dense(units = output_dense_layer_units, activation='swish')(x)
    
    if dropout:
        x = tfkl.Dropout(rate=dropout_rate)(x)
    
    x = tfkl.BatchNormalization(name='BatchNorm0')(x)
    
    
    
    # Add a Dense layer with 2 units and softmax activation as the classifier
    outputs = tfkl.Dense(2, activation='softmax')(x)

    # Create a Model connecting input and output
    tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    # Define optimizer, loss, and metrics
    # AdamW is an Adam optimizer which applies weight_decay to network layers,
    # i.e it's another way to apply l2 regularization to the whole network
    optimizer = tfk.optimizers.AdamW(1e-4, weight_decay=5e-4)
    loss = tfk.losses.CategoricalCrossentropy()
    metrics = ['accuracy']

    # Compile the model with Categorical Cross-Entropy loss and Adam optimizer
    tl_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    return tl_model

In [23]:
import keras_tuner
build_model(keras_tuner.HyperParameters())

In [24]:
#Define the tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)
tuner.search_space_summary()

Search space summary
Default search space size: 3
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout (Boolean)
{'default': False, 'conditions': []}
droprate (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.3, 'step': None, 'sampling': 'linear'}


In [25]:
#Start the search
tuner.search(X_train*255, y_train, epochs=10, validation_data=(X_val*255, y_val))

Trial 9 Complete [00h 01m 44s]
val_accuracy: 0.8057692050933838

Best val_accuracy So Far: 0.8153846263885498
Total elapsed time: 00h 15m 58s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
256               |224               |units
True              |False             |dropout
0.29004           |0.24571           |droprate

Epoch 1/10
124/124 [==============================] - 35s 107ms/step - loss: 0.7459 - accuracy: 0.6256 - val_loss: 0.6052 - val_accuracy: 0.7423
Epoch 2/10
124/124 [==============================] - 7s 54ms/step - loss: 0.6401 - accuracy: 0.6804 - val_loss: 0.5409 - val_accuracy: 0.7192
Epoch 3/10
124/124 [==============================] - 8s 67ms/step - loss: 0.6106 - accuracy: 0.7018 - val_loss: 0.4870 - val_accuracy: 0.7712
Epoch 4/10
124/124 [==============================] - 8s 67ms/step - loss: 0.5810 - accuracy: 0.7210 - val_loss: 0.4674 - val_accuracy: 0.7750
Epoch 5/10
124/124 [==============================] - 8s 67ms/ste

KeyboardInterrupt: 

In [19]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
tl_model= models[0] # Build the model.
# Needed for `Sequential` without specified `input_shape`.
tl_model.build(input_shape=(96,96,3))
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 256)              1024      
 ion)                                                        

In [20]:
# Train the model
tl_history = tl_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 64,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the EfficientNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True),
                tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=20, min_lr=1e-5, mode='max')]
).history

Epoch 1/200


KeyboardInterrupt: 

In [23]:
# Evaluate the model on the test set
test_accuracy = tl_model.evaluate(preprocess_input(X_test*255),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)
from sklearn.metrics import precision_score, recall_score
y_pred = tl_model.predict(preprocess_input(X_test*255))
y_pred = tf.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, axis=-1)
# Calculate precision and recall
precision = precision_score(y_test_true, y_pred)
recall = recall_score(y_test_true, y_pred)

# Print the precision and recall
print("Precision:", precision)
print("Recall:", recall)

Test set accuracy 0.8288
17/17 [==============================] - 5s 56ms/step
Precision: 0.8114285714285714
Recall: 0.7171717171717171


In [8]:
# Save the best model
tl_model.save('EfficientNet-AUG-FC-BATCHNORM-TL')
del tl_model

NameError: name 'tl_model' is not defined

In [9]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('EfficientNet-AUG-FC-BATCHNORM-TL')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 64)                81984     
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 64)               256       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130   

In [15]:
# Set all network layers as trainable
ft_model.get_layer('efficientnetv2-m').trainable = True

In [16]:
N = 664
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-m').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-m').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_1 False
1 rescaling False
2 stem_conv False
3 stem_bn False
4 stem_activation False
5 block1a_project_conv False
6 block1a_project_bn False
7 block1a_project_activation False
8 block1a_add False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block1c_project_conv False
15 block1c_project_bn False
16 block1c_project_activation False
17 block1c_drop False
18 block1c_add False
19 block2a_expand_conv False
20 block2a_expand_bn False
21 block2a_expand_activation False
22 block2a_project_conv False
23 block2a_project_bn False
24 block2b_expand_conv False
25 block2b_expand_bn False
26 block2b_expand_activation False
27 block2b_project_conv False
28 block2b_project_bn False
29 block2b_drop False
30 block2b_add False
31 block2c_expand_conv False
32 block2c_expand_bn False
33 block2c_expand_activation False
34 block2c_project_conv False
35 block2c_project_bn False
36 block2c_drop False
37 block2c_a

In [17]:
# Define optimizer, loss, and metrics
# AdamW is an Adam optimizer which applies weight_decay to network layers,
# i.e it's another way to apply l2 regularization to the whole network
optimizer = tfk.optimizers.AdamW(1e-4, weight_decay=5e-4)
loss = tfk.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
ft_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Display model summary
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 64)                81984     
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 64)               256       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130   

In [18]:
# Train the model
ft_history = ft_model.fit(
    x = preprocess_input(X_train*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 64,
    epochs = 200,
    validation_data = (preprocess_input(X_val*255), y_val), # We need to apply the preprocessing thought for the EfficientNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True),
                tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=20, min_lr=1e-5, mode='max')]
).history

Epoch 1/200
62/62 [==============================] - 73s 266ms/step - loss: 0.5927 - accuracy: 0.7422 - val_loss: 0.5623 - val_accuracy: 0.7404 - lr: 1.0000e-04
Epoch 2/200
62/62 [==============================] - 10s 154ms/step - loss: 0.4643 - accuracy: 0.7725 - val_loss: 0.4301 - val_accuracy: 0.8019 - lr: 1.0000e-04
Epoch 3/200
62/62 [==============================] - 9s 153ms/step - loss: 0.4176 - accuracy: 0.8027 - val_loss: 0.3880 - val_accuracy: 0.8212 - lr: 1.0000e-04
Epoch 4/200
62/62 [==============================] - 10s 154ms/step - loss: 0.4016 - accuracy: 0.8252 - val_loss: 0.3584 - val_accuracy: 0.8423 - lr: 1.0000e-04
Epoch 5/200
62/62 [==============================] - 10s 155ms/step - loss: 0.3635 - accuracy: 0.8421 - val_loss: 0.3462 - val_accuracy: 0.8500 - lr: 1.0000e-04
Epoch 6/200
62/62 [==============================] - 9s 150ms/step - loss: 0.3396 - accuracy: 0.8517 - val_loss: 0.3492 - val_accuracy: 0.8481 - lr: 1.0000e-04
Epoch 7/200
62/62 [=================

In [19]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(preprocess_input(X_test*255),y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

from sklearn.metrics import precision_score, recall_score
y_pred = ft_model.predict(preprocess_input(X_test*255))
y_pred = tf.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, axis=-1)
# Calculate precision and recall
precision = precision_score(y_test_true, y_pred)
recall = recall_score(y_test_true, y_pred)

# Print the precision and recall
print("Precision:", precision)
print("Recall:", recall)

Test set accuracy 0.8923
17/17 [==============================] - 5s 56ms/step
Precision: 0.8480392156862745
Recall: 0.8737373737373737


In [33]:
# Save the best model
ft_model.save('EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-')

In [35]:

!zip -r EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT.zip /kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-

  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/keras_metadata.pb (deflated 96%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/variables/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/variables/variables.index (deflated 77%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/saved_model.pb (deflated 92%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/assets/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-/fingerprint.pb (stored 0%)


In [28]:
import shutil
shutil.move("/kaggle/input/network-on-all-data/kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-","/kaggle/working/")
#shutil.rmtree("/kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-")

OSError: [Errno 30] Read-only file system: 'fingerprint.pb'

In [30]:
# Re-load the model after transfer learning
alldata_model = tfk.models.load_model('/kaggle/working/EFFICIENTNET-AUGGGG-FC-BATCHNORM-FT-')
alldata_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 64)                81984     
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 64)               256       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130   

In [31]:
# Set all network layers as trainable
alldata_model.get_layer('efficientnetv2-m').trainable = True

In [32]:
N = 664
for i, layer in enumerate(alldata_model.get_layer('efficientnetv2-m').layers[:N]):
  layer.trainable=False
for i, layer in enumerate(alldata_model.get_layer('efficientnetv2-m').layers):
   print(i, layer.name, layer.trainable)
alldata_model.summary()

0 input_1 False
1 rescaling False
2 stem_conv False
3 stem_bn False
4 stem_activation False
5 block1a_project_conv False
6 block1a_project_bn False
7 block1a_project_activation False
8 block1a_add False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block1c_project_conv False
15 block1c_project_bn False
16 block1c_project_activation False
17 block1c_drop False
18 block1c_add False
19 block2a_expand_conv False
20 block2a_expand_bn False
21 block2a_expand_activation False
22 block2a_project_conv False
23 block2a_project_bn False
24 block2b_expand_conv False
25 block2b_expand_bn False
26 block2b_expand_activation False
27 block2b_project_conv False
28 block2b_project_bn False
29 block2b_drop False
30 block2b_add False
31 block2c_expand_conv False
32 block2c_expand_bn False
33 block2c_expand_activation False
34 block2c_project_conv False
35 block2c_project_bn False
36 block2c_drop False
37 block2c_a

In [34]:
# Define optimizer, loss, and metrics
# AdamW is an Adam optimizer which applies weight_decay to network layers,
# i.e it's another way to apply l2 regularization to the whole network
optimizer = tfk.optimizers.AdamW(1e-5, weight_decay=5e-4)
loss = tfk.losses.CategoricalCrossentropy()
metrics = ['accuracy']

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
alldata_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Display model summary
alldata_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Functiona  (None, 1280)             53150388  
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 64)                81984     
                                                                 
 BatchNorm0 (BatchNormalizat  (None, 64)               256       
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130   

In [38]:
#train, validation, test split (80,10,10)

#one-hot encoding
y = tfk.utils.to_categorical(Y,2)

# Split data into train_val and test sets
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(X, y, random_state=seed, test_size=260, stratify=np.argmax(y,axis=1))

In [39]:
print(f"X_train shape: {X_train_final.shape}, y_train shape: {y_train_final.shape}")
print(f"X_val shape: {X_val_final.shape}, y_val shape: {y_val_final.shape}")

X_train shape: (4744, 96, 96, 3), y_train shape: (4744, 2)
X_val shape: (260, 96, 96, 3), y_val shape: (260, 2)


In [41]:
alldata_history = alldata_model.fit(
    x = preprocess_input(X_train_final*255), # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train_final,
    batch_size = 64,
    epochs = 200,
    validation_data = (preprocess_input(X_val_final*255), y_val_final), # We need to apply the preprocessing thought for the EfficientNetV2 network
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True),
                tfk.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=20, min_lr=1e-5, mode='max')]
).history

Epoch 1/200
75/75 [==============================] - 50s 216ms/step - loss: 0.1961 - accuracy: 0.9281 - val_loss: 0.1937 - val_accuracy: 0.9500 - lr: 1.0000e-05
Epoch 2/200
75/75 [==============================] - 9s 122ms/step - loss: 0.1774 - accuracy: 0.9294 - val_loss: 0.1873 - val_accuracy: 0.9500 - lr: 1.0000e-05
Epoch 3/200
75/75 [==============================] - 9s 121ms/step - loss: 0.1823 - accuracy: 0.9288 - val_loss: 0.1845 - val_accuracy: 0.9385 - lr: 1.0000e-05
Epoch 4/200
75/75 [==============================] - 9s 121ms/step - loss: 0.1691 - accuracy: 0.9342 - val_loss: 0.1831 - val_accuracy: 0.9423 - lr: 1.0000e-05
Epoch 5/200
75/75 [==============================] - 9s 121ms/step - loss: 0.1672 - accuracy: 0.9349 - val_loss: 0.1788 - val_accuracy: 0.9462 - lr: 1.0000e-05
Epoch 6/200
75/75 [==============================] - 9s 119ms/step - loss: 0.1733 - accuracy: 0.9351 - val_loss: 0.1847 - val_accuracy: 0.9423 - lr: 1.0000e-05
Epoch 7/200
75/75 [====================

In [43]:
# Save the best model
alldata_model.save('EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA')


In [44]:
!zip -r EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA.zip /kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA

  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/assets/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/keras_metadata.pb (deflated 96%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/saved_model.pb (deflated 92%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/fingerprint.pb (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/variables/ (stored 0%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/variables/variables.index (deflated 77%)
  adding: kaggle/working/EFFICIENTNET-AUG-FC-BATCHNORM-FT-ALLDATA/variables/variables.data-00000-of-00001 (deflated 8%)
